In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

## Fase de exploración

Obtenemos la página HTML desde transfermarkt de Leo Messi:

In [2]:
url = "https://www.transfermarkt.com/cristiano-ronaldo/profil/spieler/8198"
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
html = requests.get(url, headers= headers)

Importamos en BS

In [7]:
soup = BeautifulSoup(html.content, 'html.parser')

Desarrollamos la extracción del valor de mercado

In [20]:
valor = soup.find_all("div", {"class": "tm-player-market-value-development__current-value"})[0].text.strip()
valor

'€35.00m'

In [21]:
regex_valor = re.compile('\d+\.\d+')
valor = float(regex_valor.search(valor)[0])
valor

35.0

Desarrollamos la extracción del equipo

In [25]:
equipo = soup.find_all("span", {"class": "data-header__club"})[0].text.strip()
equipo

'Man Utd'

## Fase de explotación

In [26]:
jugadores = {'Lionel Messi': "https://www.transfermarkt.com/lionel-messi/profil/spieler/28003", 'Cristiano Ronaldo': "https://www.transfermarkt.com/cristiano-ronaldo/profil/spieler/8198", "Kylian Mbappé": "https://www.transfermarkt.com/kylian-mbappe/profil/spieler/342229", "Erling Haaland": "https://www.transfermarkt.com/erling-haaland/profil/spieler/418560"} 

In [27]:
jugadores["Lionel Messi"]

'https://www.transfermarkt.com/lionel-messi/profil/spieler/28003'

In [32]:
equipos = []
estimacion_mercado = []

for jugador in jugadores.keys():
    #descargamos el HTML
    url = jugadores[jugador]
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
    html = requests.get(url, headers= headers)
    
    #parseamos con bs
    soup = BeautifulSoup(html.content, 'html.parser')
    
    #obtenemos equipo
    equipo = soup.find_all("span", {"class": "data-header__club"})[0].text.strip()
    equipos.append(equipo)
    
    #obtenemos la valoracion de mercado
    valor = soup.find_all("div", {"class": "tm-player-market-value-development__current-value"})[0].text.strip()
    regex_valor = re.compile('\d+\.\d+')
    valor = float(regex_valor.search(valor)[0])
    estimacion_mercado.append(valor)

In [34]:
estimacion_mercado

[60.0, 35.0, 160.0, 150.0]

In [35]:
equipos

['Paris SG', 'Man Utd', 'Paris SG', 'Bor. Dortmund']

In [37]:
jugadores=pd.DataFrame({'Jugador': list(jugadores.keys()), 'Equipo': equipos, 'Valor_mercado_estim': estimacion_mercado})

In [38]:
jugadores

,Jugador,Equipo,Valor_mercado_estim
0,Lionel Messi,Paris SG,60.0
1,Cristiano Ronaldo,Man Utd,35.0
2,Kylian Mbappé,Paris SG,160.0
3,Erling Haaland,Bor. Dortmund,150.0
